## Juliaの浮動小数点表現を確かめる
https://docs.julialang.org/en/v1/base/numbers/#Core.Float64
```
Binary format: 1 sign, 11 exponent, 52 fraction bits.

```

In [24]:
# 64bitの浮動小数点表現の2をbitで確認
@show bitstring(Float64(1));

# -2をbitで確認、先頭ビットがたしかに符号であることがわかる
@show bitstring(Float64(-8));

# 52 fraction bitsはどうなっている？

bitstring(Float64(1)) = "0011111111110000000000000000000000000000000000000000000000000000"
bitstring(Float64(-8)) = "1100000000100000000000000000000000000000000000000000000000000000"


## Juliaのビット演算を確かめる
https://docs.julialang.org/en/v1/manual/mathematical-operations/#Bitwise-Operators

In [2]:
@show bitstring(Int64(-2));

# right logical shift
@show bitstring(Int64(-2) >>> 1);

# right arithmetic shift
@show bitstring(Int64(-2) >> 1);

bitstring(Int64(-2)) = "1111111111111111111111111111111111111111111111111111111111111110"
bitstring(Int64(-2) >>> 1) = "0111111111111111111111111111111111111111111111111111111111111111"
bitstring(Int64(-2) >> 1) = "1111111111111111111111111111111111111111111111111111111111111111"


## 上位の53bitを使って[0, 1]の浮動小数点へ変換してみる

もう１点，ビット列から double の乱数を生成する方法についての提案です．

古いメルセンヌツイスタ（https://jpn01.safelinks.protection.outlook.com/?url=http:%2F%2Fwww.math.sci.hiroshima-u.ac.jp%2F~m-mat%2FMT%2Fmt64.html&amp;data=02%7C01%7Cymatsunaga%40mail.saitama-u.ac.jp%7C18e546fd1abd421bbfbd08d85df9a8dc%7C0523af6904254a8d9827ee7292c5d821%7C1%7C0%7C637362674143135272&amp;sdata=VN08c6hwrcWQSF3BnV5kr7Q3iWbdb%2BUoplCzvDERNbY%3D&amp;reserved=0）では，
double 型 [0, 1] などの乱数を生成する場合には，

long long 型（64bit整数）の乱数を以下の方法で変換して乱数生成していたようです．

```
/* generates a random number on [0,1]-real-interval */
double genrand64_real1(void)
{
    return (genrand64_int64() >> 11) * (1.0/9007199254740991.0);
}
```

つまり 64 bit の乱数を 11 bit 右シフトして 9007199254740991.0 で割るということです．

おそらく 11 bit 右シフトした結果，残りが 53 bit なので，2^53 - 1 = 9007199254740991 で割ることで

[0, 1] の範囲に規格化しているのだと思われます．

11 bit シフトの理由は，double 型の仮数部が 52 bit であり，

"1.仮数部" の "1. " を考えると実際の精度が 2^{-53} あるためだからだと考えられます．

したがって bit 列から乱数を生成する場合には，53 bit の bit 列を用意し，

9007199254740991 で割るという処理をしてやれば良いのではないでしょうか．


In [29]:
# 適当な64bitをInt64を使って定義
b = Int64(2382938232938239322);
@show bitstring(b);

# right logical shift by 11 bits
b_shifted = b >>> 11;
@show bitstring(b_shifted);
@show b_shifted;

b_normalized = b_shifted / 9007199254740991.0; #自動的にFloat64へ変換される
@show b_normalized;

bitstring(b) = "0010000100010001111001011011001010001111111110110001010101011010"
bitstring(b_shifted) = "0000000000000100001000100011110010110110010100011111111101100010"
b_shifted = 1163544059051874
b_normalized = 0.12917934045252036


## JuliaでバイナリIOを行う

In [34]:
# 出力
io = open("test.bin", "w")

for i = 0:9
    write(io, bswap(Int64((2)^i))) #ビットが上位から順位並ぶように、bswapでバイトを逆順にしている
end

close(io)

In [35]:
# シェルコマンドを実行してファイル内容を確認。
# bswapでバイトを逆順にして、ビッグエンディアンで書き込んだので、わかりやすい。
run(`xxd -b test.bin`)

00000000: 00000000 00000000 00000000 00000000 00000000 00000000  ......
00000006: 00000000 00000001 00000000 00000000 00000000 00000000  ......
0000000c: 00000000 00000000 00000000 00000010 00000000 00000000  ......
00000012: 00000000 00000000 00000000 00000000 00000000 00000100  ......
00000018: 00000000 00000000 00000000 00000000 00000000 00000000  ......
0000001e: 00000000 00001000 00000000 00000000 00000000 00000000  ......
00000024: 00000000 00000000 00000000 00010000 00000000 00000000  ......
0000002a: 00000000 00000000 00000000 00000000 00000000 00100000  ..... 
00000030: 00000000 00000000 00000000 00000000 00000000 00000000  ......
00000036: 00000000 01000000 00000000 00000000 00000000 00000000  .@....
0000003c: 00000000 00000000 00000000 10000000 00000000 00000000  ......
00000042: 00000000 00000000 00000000 00000000 00000001 00000000  ......
00000048: 00000000 00000000 00000000 00000000 00000000 00000000  ......
0000004e: 00000010 00000000                                     

Process(`xxd -b test.bin`, ProcessExited(0))

In [36]:
# 入力
io = open("test.bin", "r")

seekend(io)
file_size_in_8bit = position(io) # get file size in byte
file_size_in_64bit = Int(file_size_in_8bit/8)
seekstart(io)

d_array = Array{Int64}(undef, 0)
for i = 1:file_size_in_64bit
    d = bswap(read(io, Int64)) #入力のときもbswapでバイトを逆順にする
    push!(d_array, d)
end

close(io)

d_array

10-element Array{Int64,1}:
   1
   2
   4
   8
  16
  32
  64
 128
 256
 512

## 読み込んだ64bit単位の各上位53bitを使って[0,1]の浮動小数点へ変換

In [7]:
rand_array = similar(d_array, Float64)
for i in 1:length(d_array)
    d = d_array[i]
    d_shifted = d >>> 11
    rand_array[i] = d_shifted / 9007199254740991.0 #自動的にFloat64へ変換される
end

rand_array

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 2.2204460492503136e-16
 1.776356839400251e-15
 1.4210854715202007e-14
 1.1368683772161605e-13
 9.094947017729284e-13
 7.275957614183428e-12
 5.820766091346742e-11

In [ ]:
# FORTRAN専用のバイナリファイルとして出力